# Plotting Images and Bounding Boxes

* It supports both the true and the predicted bounding boxes.

## Boxes in YXHW format

In [ ]:
def plot_boxes(ax, boxes, size, color='green', alpha=1.0, fill=False, no_edge=False, topLabel=True):
    """
        ax: Plot Axes.
        boxes: A tensor of YXHW values for the boxes. Shape: (N_BOXES, 4)
        size: Size of the image. It is used to translate the YXHW values to image size.
        color: A string to indicate the color in matplotlib format for the bounding boxes.
        alpha: A FP value for the box fill.
        fill: If true, fill the boxes with color.
        no_edge: If true, we do not draw the box edges.
        topLabel: If true, add the box id at the top-left corner of the box.
            If false, it goes to the bottom-right corner.
    """
    boxes = boxes*size
    # tf.print(boxes)

    for box_id, box in enumerate(boxes):
        [y_min, x_min, height, width] = box
        rect = patches.Rectangle([x_min, y_min], width, height, ec=None if no_edge else color, fill=fill, fc=color, alpha=alpha)
        ax.add_patch(rect)
        text_bbox = dict(ec=color, fc=color, mutation_aspect=.05, boxstyle=patches.BoxStyle.Square(pad=.4))

        if topLabel:
            x, y = rect.get_xy()
            x += 4
            ha, va = 'left', 'bottom'
        else:
            x, y = x_min + width - 4, y_min + height 
            ha, va = 'right', 'bottom'
        
        ax.text(x, y, str(box_id + 1), ha=ha, va=va, size='xx-small', color='white', bbox=text_bbox)

def plot_examples(images, y_true, y_pred, cols=3, match=False):
    """
        It plots true and predicted bounding boxes over the images.

        Arguments:
            images: A tensor of images. Shape: (BATCH_SIZE, IMG_SIZE, IMG_SIZE, 3)
            y_true: A grid of box HW values for each feature point. Shape: (BATCH_SIZE, SIZE, SIZE, 2)
            y_pred: A tensor of YXHW values for predicted boxes. Shape: (N_BOXES, 4)
            cols: The number of columns in the plot.
            match: If true, we truncate the predicted boxes to the number of true boxes.
    """
    batch_size = y_pred.shape[0] if y_pred is not None else y_true.shape[0]

    # Plot configuration
    rows = (batch_size + cols - 1)//cols
    fig, axes = plt.subplots(rows, cols, figsize=(8, rows*3))
    color_1, color_2, color_i, color_n = ['red', 'green', 'blue', 'yellow']
    axes = axes.ravel()

    def slice_ys(item_id):
        item_y_true = None if y_true is None else hw_grid_to_yxhw(y_true[item_id])

        num_y_pred_boxes = tf.shape(item_y_true)[0] if match and item_y_true is not None else MAX_BOXES
        item_y_pred = None if y_pred is None else y_pred[item_id, :num_y_pred_boxes]

        return item_y_true, item_y_pred

    for item_id in range(batch_size):
        # Plot image
        axes[item_id].imshow(images[item_id])

        item_y_true, item_y_pred = slice_ys(item_id)

        # Plot predicted boxes
        plot_boxes(axes[item_id], item_y_pred if item_y_pred is not None else [], IMG_SIZE, color_2, topLabel=False)
        plot_boxes(axes[item_id], item_y_true if item_y_true is not None else [], IMG_SIZE, color_1)

        # Add box counts as plot title.
        title = ' | '.join(
            map(
                lambda x: str(int(tf.shape(x)[0])),
                filter(
                    lambda x: x is not None,
                    [item_y_pred, item_y_true]
                )
            )
        )
        axes[item_id].set_title(title)

    plt.tight_layout()

# itr = iter(train_prep_ds.batch(2))
# images, y_true = next(itr)

# plot_examples(images, y_true, None)

# r = patches.Rectangle([0, 0], 5, 5)

## Boxes in CYCX format

# History Plots

In [ ]:
def plot_metrics(ax, h, names, labels, title='Plot', x_label='Epoch', y_label=None):
    val_name = lambda name: 'val_{}'.format(name)
    val_label = lambda label: 'Validation {}'.format(label)

    # Plot metric values on left y-axis
    for name, label in zip(names, labels):
        metric = h[name]
        ax.plot(range(1, len(metric) + 1), metric, label=label)
        ax.plot(range(1, len(metric) + 1), h[val_name(name)], label=val_label(label))
    
    # Color plot background to indicate learning rate changes.
    for index, lr_value in enumerate(h['learning_rate'][:-1]):
        ax.axvspan(index, index + 1, fc='#77d8c0', alpha=(1 - (1/EPOCHS)*(index)))
    
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.legend()

    # Learning rates are plotted on the right y-axis
    right_ax = ax.twinx()
    right_ax.plot(h['learning_rate'], label='Epoch LR', ls='-.')

    right_ax.set_ylabel('Learning Rate')
    right_ax.legend()

# plot_metrics(None, hist.history, ['loss', 'yx_loss'], ['Loss', 'YX Loss'], 'Losses')

def plot_history(h):
    fig, axes = plt.subplots(2, 2, figsize=(12, 8), facecolor='w', edgecolor='k')
    axes = axes.ravel()

    loss, iou, positive_iou, negative_iou = axes

    # Plot loss metrics
    plot_metrics(loss, h, ['loss', 'yx_loss', 'hw_loss'], ['Loss', 'YX Loss', 'HW Loss'], title='Losses', y_label='Loss')
    
    # Plot IoU metrics
    plot_metrics(iou, h, ['iou'], ['IoU'], title='IoUs', y_label='IoU')

    # Plot positive IoU metrics
    plot_metrics(positive_iou, h, ['positive_iou'], ['Positive IoU'], title='Positive IoUs', y_label='IoU')

    # Plot negative IoU metrics
    plot_metrics(negative_iou, h, ['negative_iou'], ['Negative IoU'], title='Negative IoUs', y_label='IoU')

    plt.tight_layout()

plot_history(hist.history)
hist.history